# Monte Carlo class usage

![Landing point dispersion ellipses](https://raw.githubusercontent.com/RocketPy-Team/RocketPy/master/docs/notebooks/monte_carlo_analysis/monte_carlo_analysis_outputs/valetudo_rocket_v0.svg)

This is an advanced use of RocketPy. This notebook runs a Monte Carlo analysis and predicts probability distributions of the rocket's landing point, apogee and other relevant information. 

The `MonteCarlo` class simplifies the process of performing Monte Carlo simulations. The idea is to take the already defined classes for a standard flight simulation, and create the so called ``Stochastic`` classes, which are used to run the Monte Carlo analysis.

This class offers extensive capabilities, and this example notebook covers as many as possible.
For a deeper understanding, we recommend checking the class [documentation](https://docs.rocketpy.org/en/latest/).

For a more comprehensive conceptual understanding of Monte Carlo Simulations, refer to RocketPy's main reference: [RocketPy: Six Degree-of-Freedom Rocket Trajectory Simulator](https://doi.org/10.1061/(ASCE)AS.1943-5525.0001331).



In [ ]:
# We import these lines for debugging purposes, only works on Jupyter Notebook
%load_ext autoreload
%autoreload 2

First, let's import the necessary libraries

In [ ]:
from rocketpy import Environment, SolidMotor, Rocket, Flight, MonteCarlo, Function
from rocketpy.stochastic import (
    StochasticEnvironment,
    StochasticSolidMotor,
    StochasticRocket,
    StochasticFlight,
    StochasticNoseCone,
    StochasticTail,
    StochasticTrapezoidalFins,
    StochasticParachute,
    StochasticRailButtons,
)
import datetime

If you are using Jupyter Notebooks, it is recommended to run the following line to make matplotlib plots which will be shown later interactive and higher quality.


In [ ]:
%matplotlib widget

## Step 1: Standard Simulation


We will first create a standard RocketPy simulation objects (e.g. Environment, SolidMotor, etc.) to then create the Stochastic objects.

The standard objects created here are the same as in the [First Simulation Page](https://docs.rocketpy.org/en/latest/user/first_simulation.html) of our documentation, so you can go through that if you want to understand what is done in more detail.

The only difference here is that we will use a `Environment` with atmospheric model type `Ensemble`. This allows us to run the Monte Carlo analysis with different ensemble members, which are different atmospheric profiles.

We will do it all in one single cell for simplicity.

In [ ]:
# Environment
env = Environment(latitude=39.389700, longitude=-8.288964, elevation=113)
tomorrow = datetime.date.today() + datetime.timedelta(days=1)
env.set_date((tomorrow.year, tomorrow.month, tomorrow.day, 12))
env.set_atmospheric_model(type="Ensemble", file="GEFS")

# Motor
motor = SolidMotor(
    thrust_source="../../../data/motors/Cesaroni_M1670.eng",
    dry_mass=1.815,
    dry_inertia=(0.125, 0.125, 0.002),
    nozzle_radius=33 / 1000,
    grain_number=5,
    grain_density=1815,
    grain_outer_radius=33 / 1000,
    grain_initial_inner_radius=15 / 1000,
    grain_initial_height=120 / 1000,
    grain_separation=5 / 1000,
    grains_center_of_mass_position=0.397,
    center_of_dry_mass_position=0.317,
    nozzle_position=0,
    burn_time=3.9,
    throat_radius=11 / 1000,
    coordinate_system_orientation="nozzle_to_combustion_chamber",
)
print(f"Total Impulse of the Solid Motor: {motor.total_impulse} Ns")

# Rocket
rocket = Rocket(
    radius=127 / 2000,
    mass=14.426,
    inertia=(6.321, 6.321, 0.034),
    power_off_drag="../../../data/calisto/powerOffDragCurve.csv",
    power_on_drag="../../../data/calisto/powerOnDragCurve.csv",
    center_of_mass_without_motor=0,
    coordinate_system_orientation="tail_to_nose",
)

rail_buttons = rocket.set_rail_buttons(
    upper_button_position=0.0818,
    lower_button_position=-0.618,
    angular_position=45,
)

rocket.add_motor(motor, position=-1.255)

nose_cone = rocket.add_nose(length=0.55829, kind="vonKarman", position=1.278)

fin_set = rocket.add_trapezoidal_fins(
    n=4,
    root_chord=0.120,
    tip_chord=0.060,
    span=0.110,
    position=-1.04956,
    cant_angle=0.5,
    airfoil=("../../../data/calisto/NACA0012-radians.csv", "radians"),
)

tail = rocket.add_tail(
    top_radius=0.0635, bottom_radius=0.0435, length=0.060, position=-1.194656
)
Main = rocket.add_parachute(
    "Main",
    cd_s=10.0,
    trigger=800,
    sampling_rate=105,
    lag=1.5,
    noise=(0, 8.3, 0.5),
)

Drogue = rocket.add_parachute(
    "Drogue",
    cd_s=1.0,
    trigger="apogee",
    sampling_rate=105,
    lag=1.5,
    noise=(0, 8.3, 0.5),
)

# Flight
test_flight = Flight(
    rocket=rocket,
    environment=env,
    rail_length=5,
    inclination=84,
    heading=133,
)

Lets check the trajectory of the Flight.

In [ ]:
test_flight.plots.trajectory_3d()

The flight trajectory above represents the nominal trajectory of the rocket, without any uncertainties.


## Step 2: Stochastic Objects

For each RocketPy object, we will create a ``Stochastic`` counterpart that extends the initial model, allowing us to define the uncertainties of each input parameter.

Please refer to the [Working with Stochastic Objects](https://docs.rocketpy.org/en/latest/user/stochastic.html#working-with-stochastic-objects) page on RocketPy`s documentation for a more detailed explanation.

### Stochastic Environment


Starting with the `Environment` object, we will create a `StochasticEnvironment` to specify its uncertainties.

In this first example, we will specify the ensemble member and wind velocities factor.

Since the ensemble member is a discrete value, **only list type inputs are permitted**. The list will contain the ensemble numbers to be randomly selected during the Monte Carlo simulation. This means that in each iteration, a different ensemble member will be chosen.

<!-- The wind velocities factor are also special inputs. They are used to scale the wind velocities in each axis. The factor inputs can only be tuples or lists, since it has no nominal value to get from the standard. Lets scale the wind by a factor of 1.00000 ± 0.2 -->

In [ ]:
stochastic_env = StochasticEnvironment(
    environment=env,
    ensemble_member=list(range(env.num_ensemble_members)),
)

stochastic_env.visualize_attributes()

#### NOTE

Always check the `.visualize_attributes()` method of each stochastic object to verify the uncertainties were correctly set.


Just to illustrate the potential of this technique, let's randomly generate 5 instances of the environment using the `create_object` method.

For each instance, we will calculate the wind speed at 1km altitude and store the results in a list.

In [ ]:
wind_speed_at_1000m = []
for i in range(5):
    rnd_env = stochastic_env.create_object()
    wind_speed_at_1000m.append(rnd_env.wind_velocity_x(1000))

print(wind_speed_at_1000m)

As you can see, the wind speed varies between ensemble members.
This demonstrates how the Monte Carlo simulation can capture the variability in wind conditions due to different ensemble members.


### Motor


We can now create a `StochasticSolidMotor` object to define the uncertainties associated with the motor.
In this example, we will apply more complex uncertainties to the motor parameters.

The `StochasticSolidMotor` also has one special parameter which is the `total_impulse`. It lets us alter the total impulse of the motor while maintaining the thrust curve shape. This is particularly useful for motor uncertainties.

In [ ]:
stochastic_motor = StochasticSolidMotor(
    solid_motor=motor,
    burn_start_time=(0, 0.1, "binomial"),
    grains_center_of_mass_position=0.001,
    grain_density=50,
    grain_separation=1 / 1000,
    grain_initial_height=1 / 1000,
    grain_initial_inner_radius=0.375 / 1000,
    grain_outer_radius=0.375 / 1000,
    total_impulse=(6500, 1000),
    throat_radius=0.5 / 1000,
    nozzle_radius=0.5 / 1000,
    nozzle_position=0.001,
)
stochastic_motor.visualize_attributes()

#### NOTE

Pay special attention to how different input types are interpreted in the `StochasticSolidMotor` object by checking the printed object:


<!-- - ``thrust_source`` was given as a list of 3 items, and is saved as is. This means that the simulation will randomly chose one item of that list, as desired -->

- ``burn_start_time`` was given as a tuple of 3 items, specifying the nominal value, the standard deviation and the distribution type

- ``total_impulse`` was given as a tuple of 2 numbers, so the distribution type was set to the default: `normal`

- All other values set for the other parameters in the constructor are simple values, which means they are interpreted as standard deviation and the nominal value is taken from the ``motor``

- The remaining parameters that are printed are just the nominal values from the ``motor``. In the ``Stochastic`` object they are saved as a list of one item



Once again, we can illustrate the power of stochastic modeling by generating multiple instances of the `SolidMotor` class using the `StochasticSolidMotor` object.
For each instance, we will calculate the total impulse and store the results in a list. This will show how the uncertainties in the motor parameters affect the total impulse over multiple iterations.


In [ ]:
total_impulse = []
for i in range(5):
    rnd_motor = stochastic_motor.create_object()
    total_impulse.append(rnd_motor.total_impulse)

print(total_impulse)

### Rocket


We can now create a `StochasticRocket` object to define the uncertainties associated with the rocket.

In [ ]:
stochastic_rocket = StochasticRocket(
    rocket=rocket,
    radius=0.0127 / 2000,
    mass=(15.426, 0.5, "normal"),
    inertia_11=(6.321, 0),
    inertia_22=0.01,
    inertia_33=0.01,
    center_of_mass_without_motor=0,
)
stochastic_rocket.visualize_attributes()

The `StochasticRocket` still needs to have its aerodynamic surfaces and parachutes added. 

We can also create stochastic models for each aerodynamic surface, although this is not mandatory.

In [ ]:
stochastic_nose_cone = StochasticNoseCone(
    nosecone=nose_cone,
    length=0.001,
)

stochastic_fin_set = StochasticTrapezoidalFins(
    trapezoidal_fins=fin_set,
    root_chord=0.0005,
    tip_chord=0.0005,
    span=0.0005,
)

stochastic_tail = StochasticTail(
    tail=tail,
    top_radius=0.001,
    bottom_radius=0.001,
    length=0.001,
)

stochastic_rail_buttons = StochasticRailButtons(
    rail_buttons=rail_buttons, buttons_distance=0.001
)

stochastic_main = StochasticParachute(
    parachute=Main,
    cd_s=0.1,
    lag=0.1,
)

stochastic_drogue = StochasticParachute(
    parachute=Drogue,
    cd_s=0.07,
    lag=0.2,
)

Then we must add them to our stochastic rocket, much like we do in the normal Rocket.



In [ ]:
stochastic_rocket.add_motor(stochastic_motor, position=0.001)
stochastic_rocket.add_nose(stochastic_nose_cone, position=(1.134, 0.001))
stochastic_rocket.add_trapezoidal_fins(stochastic_fin_set, position=(0.001, "normal"))
stochastic_rocket.add_tail(stochastic_tail)
stochastic_rocket.set_rail_buttons(
    stochastic_rail_buttons, lower_button_position=(0.001, "normal")
)
stochastic_rocket.add_parachute(stochastic_main)
stochastic_rocket.add_parachute(stochastic_drogue)

#### NOTE

The `position` arguments behave just like the other ``Stochastic`` classes parameters



Now lets check how the `StochasticRocket` handled all these additions

In [ ]:
stochastic_rocket.visualize_attributes()


### Flight


After defining the `Flight`, we can create the corresponding `Stochastic` object to define the uncertainties of the input parameters.

In [ ]:
stochastic_flight = StochasticFlight(
    flight=test_flight,
    inclination=(84.7, 1),  # mean= 84.7, std=1
    heading=(53, 2),  # mean= 53, std=2
)
stochastic_flight.visualize_attributes()

### Step 2: Starting the Monte Carlo Simulations


First, let's invoke the `MonteCarlo` class, we are going to need a filename to initialize it.
The filename will be used either to save the results of the simulations or to load them
from a previous ran simulation.


In [ ]:
test_dispersion = MonteCarlo(
    filename="monte_carlo_analysis_outputs/monte_carlo_class_example",
    environment=stochastic_env,
    rocket=stochastic_rocket,
    flight=stochastic_flight,
)

Finally, let's simulate our flights. 
We can run the simulations using the method `MonteCarlo.simulate()`.

Set `append=False` to overwrite the previous results, or `append=True` to add the new results to the previous ones.


In [ ]:
test_dispersion.simulate(number_of_simulations=1000, append=False)

### Visualizing the results


Now we finally have the results of our Monte Carlo simulations loaded!
Let's play with them.


First, we can print numerical information regarding the results of the simulations.


In [ ]:
# You only need to import results if you did not run the simulations
# test_dispersion.import_results()

In [ ]:
test_dispersion.num_of_loaded_sims

In [ ]:
test_dispersion.prints.all()

Secondly, we can plot the results of the simulations.


In [ ]:
test_dispersion.plots.ellipses(xlim=(-200, 3500), ylim=(-200, 3500))

In [ ]:
test_dispersion.plots.all()

Finally, one may also export the ellipses to a ``.kml`` file so it can be easily visualized in Google Earth


In [ ]:
test_dispersion.export_ellipses_to_kml(
    filename="monte_carlo_analysis_outputs/monte_carlo_class_example.kml",
    origin_lat=env.latitude,
    origin_lon=env.longitude,
    type="impact",
)